In [1]:
import numpy as np
import gym
import random
from tqdm import tqdm

In [2]:
%%html
<img src="img/eps-greedy.png">
<img src="img/algo.png">

In [3]:
env = gym.make('FrozenLake-v0')
alpha = 0.1
max_steps = 250 
episodes = 100000
gamma = 0.95
epsilon = 0.8
_lambda = 0.8

In [4]:
def Initialize_Q():
    Q = np.zeros((env.env.nS,env.env.nA))
    return Q

def Initialize_E():
    E = np.zeros((env.env.nS,env.env.nA))
    return E

def Initalize_Policy(env):
    Policy = np.array([env.action_space.sample() for _ in range(env.env.nS)])
    return Policy

def tune_params():
    global epsilon, alpha
    epsilon = epsilon * 0.99
    alpha = alpha * 0.9995

def eps_greedy_policy(state,Q):
    if random.uniform(0,1) <=  1 - epsilon:
        new_action = np.argmax(Q[state,:]) #exploit
    else:
        new_action = env.action_space.sample() #explore
    return new_action
    
def Sarsa_lambda(episodes=episodes,max_steps=max_steps):
    Q = Initialize_Q()
    total_steps = 0

    for k in tqdm(range(episodes)):
        
        env.reset() #not sure here
        
        #init E,S,A
        E = Initialize_E()
        state = random.randint(0,env.env.nS-1) #env.reset() 
        action = eps_greedy_policy(state,Q)
        
        for step in range(max_steps):
            #Take action A, ovserve R,S'
            new_state, reward, done, _ = env.step(action)
            new_action = eps_greedy_policy(new_state,Q)
            
            delta_error = reward + gamma*Q[new_state,new_action]-Q[state,action]
            E[state,action]+=1
            Q = np.add(Q, np.multiply(alpha * delta_error, E))
            E = np.multiply(gamma * _lambda, E)
            
            state=new_state
            action=new_action
            if done:
                #decay epsilon and alpha each episode (maybe will improve results)
                #tune_params() 
                break
        
        total_steps+=step
        # Do here something every number of steps:
        # if total_steps%< == 0:
        
    return Q,np.argmax(Q,axis=1) #returns Q and the policy


In [5]:
Q,Policy = Sarsa_lambda(episodes,max_steps)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:19<00:00, 5021.32it/s]


In [6]:
def decode_action(action):
    if action == 0:
        return "L"
    if action == 1:
        return "D"
    if action == 2:
        return "R"
    if action == 3:
        return "U"
    else:
        return "Unknown"

env.reset()
env.render()
print(np.array([decode_action(action) for action in Policy]).reshape((4,4)))


SFFF
FHFH
FFFH
HFFG
[['D' 'D' 'L' 'L']
 ['R' 'R' 'L' 'D']
 ['R' 'U' 'L' 'L']
 ['D' 'R' 'R' 'D']]
